### 지갑주소 생성

In [2]:
import bitcoin.main as btc
# 개인키를 생성한다
while (1):
    privKey = btc.random_key()
    dPrivKey = btc.decode_privkey(privKey, 'hex')
    if dPrivKey < btc.N:
        break
# 개인키로 공개키를 생성
pubKey = btc.privkey_to_pubkey(privKey)

# 공개키로 지갑 주소를 생성(mainnet 용)
addressmain = btc.pubkey_to_address(pubKey, 0)

# 공개키로 지갑 주소를 생성(testnet 용)
addresstest = btc.pubkey_to_address(pubKey, 0x6f)

# 결과 확인
print("\n개인키 : ", privKey)
print("\n공개키 : ", pubKey)
print("\n지갑주소 (mainnet 용) : ", addressmain)
print("\n지갑주소 (Testnet 용) : ", addresstest)


개인키 :  5a71369d7ce22cb95aeacd987fd32396f8ae0136fb17c52227dc6110d332abda

공개키 :  04981c13c53bc0415407ab9ffc9c0a607807fd5d33e99f5696229c12eb32bb9f372ae983158a25ae4e553a66af6aa9049f7fceae78011ff6f89ef7cec1aedc3907

지갑주소 (mainnet 용) :  1DyHmpn8oJCrXjWb5DNTUmYd57fXRKZ5iL

지갑주소 (Testnet 용) :  mtVF4ss7cKe7JqzCnnLqJgkww7GEPZ7Njr


### Transaction

In [3]:
from bitcoin.bci import history
from bitcoin.transaction import mktx, sign, deserialize
from urllib.request import urlopen
from urllib.parse import urlencode
url = "https://testnet.blockchain.info/"
A = 0   # 첫번째 Address
B = 1   # 두번째 Address
address = ['mzjRhagRbxKZ9nLiEa8Fjbptj78Do4A2am',
           'mtVF4ss7cKe7JqzCnnLqJgkww7GEPZ7Njr']
privKey = ['4c7d6bcb5fbc452a2cdeedd95b9bd31b5dca6f8974b2850a3ac6d6efa132622f',
           '5a71369d7ce22cb95aeacd987fd32396f8ae0136fb17c52227dc6110d332abda']

In [5]:
def getUtxo(n=A):
    if n == A or n == B:
        h = history(address[n])
        return list(filter(lambda txo: 'spend' not in txo, h))
    else:
        print("address error.")
        
getUtxo(A)
#getUtxo(B)

[{'address': 'mzjRhagRbxKZ9nLiEa8Fjbptj78Do4A2am',
  'value': 5372307,
  'output': 'e1fca838a24faaaa45f52b4857a764a9388ff3676c63b8d5c3dcdc2741a2aa97:0',
  'block_height': None}]

In [6]:
def createTx(utxo, n1, n2, value, fee):
    # Input을 생성
    totalValue = 0
    inputs = []
    for i in range(len(utxo)):
        totalValue += utxo[i]['value'] * 1e-8
        inputs.append(utxo[i])
        if totalValue > (value + fee):
            break
            
    # 수수료를 차감한 거스름돈을 계산
    outChange = totalValue - value - fee
    if outChange < 0:
        raise ValueError("Value is larger than input amount")
    if outChange > 0:
        chgSatoshi = int(outChange * 1e8)
        change = [{'value': chgSatoshi, 'address': address[n1]}]
        
    else:
        change = []

    # Output을 생성
    outputs = [{'value': int(value * 1e8), 'address': address[n2]}]
    
    # transaction을 만듦
    tx = mktx(inputs,outputs+change)
    return tx, len(inputs)

def signTx(tx, nInput, n1):
    for i in range(nInput):
        tx = sign(tx, i, privKey[n1])
        return tx

utxo = getUtxo(A)
tx,ninput = createTx(utxo, A, B, 0.001, 0.0001)
tx = signTx(tx, ninput, A)
tx

'010000000197aaa24127dcdcc3d5b8636c67f38f38a964a757482bf545aaaa4fa238a8fce1000000008b483045022100bcb4c3bb270e8605b4e4e091a110e86675a56f77c9d758d1161f2adb7b01a56b022066805a59467c6c10affc28e16f2309c44a34476b900bb534d2b510d13f462415014104f40525d54ce74ae109114375b41063f3acd5b16216b661e2b3980c2bdb5fbfa4f45d283460b6295eff16320e1d5d721f7df68b1eebf0fc2f30c73c77ccf99bbaffffffff02a0860100000000001976a9148e46cdf4984bd7a055a34c4bb66ad6907583dab388ace24b5000000000001976a914d2c6422e03b9073206a0215d41c99c6a438b135b88ac00000000'

In [ ]:
def sendTx(tx):
    params = {'tx': tx}
    payload = urlencode(params).encode('utf-8')
    response = urlopen(url + 'pushtx', payload).read()
    print(response.decode('utf-8'))
